In [1]:
from pathlib import Path
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import json

## Dataset

### MNIST

In [3]:
training_data = datasets.MNIST(
    root="./benchmark/mnist/data/",
    train=True,
    download=False,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
)

testing_data = datasets.MNIST(
    root="./benchmark/mnist/data/",
    train=False,
    download=False,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
)

### EMNIST

In [2]:
training_data = datasets.EMNIST(
    root="./benchmark/emnist/data/",
    split="letters",
    train=True,
    download=False,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
)

testing_data = datasets.EMNIST(
    root="./benchmark/emnist/data/",
    split="letters",
    train=False,
    download=False,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
)

### CIFAR10

In [13]:
training_data = datasets.CIFAR10(
    root="./benchmark/cifar10/data/",
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

testing_data = datasets.CIFAR10(
    root="./benchmark/cifar10/data/",
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./benchmark/cifar10/data/cifar-10-python.tar.gz to ./benchmark/cifar10/data/
Files already downloaded and verified


### Fashion MNIST

In [29]:
training_data = datasets.FashionMNIST(
    root="./benchmark/fmnist/data", 
    train=True, 
    download=True, 
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
)

testing_data = datasets.FashionMNIST(
    root="./benchmark/fmnist/data", 
    train=False, 
    download=True, 
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./benchmark/fmnist/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./benchmark/fmnist/data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./benchmark/fmnist/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./benchmark/fmnist/data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./benchmark/fmnist/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./benchmark/fmnist/data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./benchmark/fmnist/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./benchmark/fmnist/data/FashionMNIST/raw



### CIFAR100

In [2]:
training_data = datasets.CIFAR100(
    root="./benchmark/cifar100/data/",
    train=True,
    download=False,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

testing_data = datasets.CIFAR100(
    root="./benchmark/cifar100/data/",
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

Files already downloaded and verified


## Division algorithm

**Each client:**

1. Contains no more than 3 labels
2. Each label has 8 to 20 samples
3. There're at least 5 * #numclass clients

In [30]:
import numpy as np
import os
import math

In [35]:
total_labels = np.unique(training_data.targets).tolist()
len(total_labels)
print(total_labels)

min_label_per_client = 2
max_label_per_client = 4

min_sample_per_client = 10
max_sample_per_client = 20

# num_clients = 5 * len(total_labels)
num_clients = 500

total_label = len(total_labels)
label_list = [i for i in total_labels]
label_per_client = 2

labels = training_data.targets
idxs = range(len(training_data))
training_idxs_labels = np.vstack((idxs, labels)).T

labels = testing_data.targets
idxs = range(len(testing_data))
testing_idxs_labels = np.vstack((idxs, labels)).T

training_dict_client = {client_id:[] for client_id in range(num_clients)}
testing_dict_client = {client_id:[] for client_id in range(num_clients)}

client_labels = []
not_passed_label_list = label_list.copy()

for client_id in range(num_clients):
    label_per_client = np.random.randint(min_label_per_client, max_label_per_client + 1)
    this_set = np.random.choice(label_list, label_per_client, replace=False)
    client_labels.append(list(this_set))
    not_passed_label_list = list(set(not_passed_label_list) - set(this_set))

if len(not_passed_label_list) > 0:
    print("Uncover", len(not_passed_label_list), "labels !")
    exit(0)
else:
    print("Uncover", len(not_passed_label_list), "labels !")

samples_details = []

for client_idx, client_label in zip(range(num_clients), client_labels):
    sample_this_client = []
    
    for label in client_label:
        sample_per_client = np.random.randint(min_sample_per_client, max_sample_per_client + 1)
        sample_this_client.append(sample_per_client)
        
        idxes_1 = training_idxs_labels[training_idxs_labels[:,1] == label][:,0]
        idxes_2 = testing_idxs_labels[testing_idxs_labels[:,1] == label][:,0]
        
        label_1_idxes = np.random.choice(idxes_1, sample_per_client, replace=False)
        label_2_idxes = np.random.choice(idxes_2, int(sample_per_client/4), replace=False)
        
        training_dict_client[client_idx] += label_1_idxes.tolist()
        testing_dict_client[client_idx] += label_2_idxes.tolist()
        
        training_idxs_labels[label_1_idxes] -= 100
        testing_idxs_labels[label_2_idxes] -= 100
    
    samples_details.append(sample_this_client)


dis_mtx = np.zeros([num_clients, total_label])
for client_id in range(len(client_labels)):
    client_label = client_labels[client_id]
    client_samples = samples_details[client_id]
    
    for label, num_samples in zip(client_label, client_samples):
        dis_mtx[client_id][total_labels.index(label)] = num_samples

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Uncover 0 labels !


In [34]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
savepath = f"./dataset_idx/fmnist/sparse/{num_clients}client"
if not Path(savepath).exists():
    os.makedirs(savepath)
    
json.dump(training_dict_client, open(f"{savepath}/fmnist_sparse.json", "w"), cls=NumpyEncoder)
json.dump(testing_dict_client, open(f"{savepath}/fmnist_sparse_test.json", "w"), cls=NumpyEncoder)
np.savetxt(f"{savepath}/fmnist_sparse_stat.csv", dis_mtx, fmt="%d", delimiter=",")

### Gen I.i.d data

In [13]:
num_clients = 30
sample_per_client = 200

training_dict_client = {client_id:[] for client_id in range(num_clients)}
testing_dict_client = {client_id:[] for client_id in range(num_clients)}

labels = training_data.targets
idxs = range(len(training_data))
training_idxs_labels = np.vstack((idxs, labels)).T

labels = testing_data.targets
idxs = range(len(testing_data))
testing_idxs_labels = np.vstack((idxs, labels)).T


for client_id in range(num_clients):    
    idxes_1 = training_idxs_labels[:,0]
    idxes_2 = testing_idxs_labels[:,0]
    
    label_1_idxes = np.random.choice(idxes_1, sample_per_client, replace=False)
    label_2_idxes = np.random.choice(idxes_2, int(sample_per_client/4), replace=False)
    
    training_dict_client[client_id] += label_1_idxes.tolist()
    testing_dict_client[client_id] += label_2_idxes.tolist()
    
    training_idxs_labels[label_1_idxes] -= 100
    testing_idxs_labels[label_2_idxes] -= 100

In [12]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
savepath = f"./dataset_idx/mnist/iid/{num_clients}client"
if not Path(savepath).exists():
    os.makedirs(savepath)
    
json.dump(training_dict_client, open(f"{savepath}/mnist_iid.json", "w"), cls=NumpyEncoder)
json.dump(testing_dict_client, open(f"{savepath}/mnist_iid_test.json", "w"), cls=NumpyEncoder)